In [8]:
# libraries

import matplotlib.pyplot as plt  
import numpy as np
import cv2
import pickle
import os, sys
from IPython.display import display



In [9]:
import tkinter as tk
from tkinter import*
from tkinter import filedialog
from tkinter.filedialog import askopenfilenames
from PIL import ImageTk, Image
import os
global filen

In [10]:
def adaptive_gaussian_thresholding(image):
    
    row,col = image.shape
    padded_img = np.zeros((row+2,col+2),dtype = np.uint8)
    thresholded_img = np.zeros((row,col),dtype = np.uint8)
    filter = [[1,4,7,4,1],[4,16,26,16,4],[7,26,41,26,7],[4,16,26,16,4],[1,4,7,4,1]]
    
    for i in range(row):
        for j in range(col):
            padded_img[i+1,j+1] = image[i,j]
        
    for i in range(row):
        for j in range(col):
            
            temp1 = 0
            mean = 0
            l = i
            m = j

            # 5*5 window
            for x in range(4):
                for y in range(4):
                    if(padded_img[l+x-1,m+x-1] != 0):
                        temp1 += (padded_img[l+x-1,m+y-1]*filter[x][y])
                                                 
                         
            mean = temp1/255 

            if(image[i,j] >= mean):
                thresholded_img[i,j] = 0
            else:
                thresholded_img[i,j] = 255
           
    
    return thresholded_img 

In [11]:
def detectionResult():
    # variables
    images = []
    resized = []
    gray = []
    edge = []
    threashold_image = []
    path=root.filen[0]
    img = cv2.imread(path)
    img2 = img.copy()
    images.append(img)

    # resizing images
    #resized_img = cv2.resize(img,(500,500))
    #resized.append(resized_img)

    grayImage = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
    gray.append(grayImage)

    # calculating edges using canny edge detection
    new_img = (grayImage*255).astype(np.uint8)
    edges = cv2.Canny(new_img,100,200)
    edge.append(edges)

    # gausian thresholding
    thres = adaptive_gaussian_thresholding(edges)
    threashold_image.append(thres)

    # binarize the image
    bin = cv2.threshold(thres, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

    # performing morphological operations 
    # construct a closing kernel containing all ones to perform operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))          ## np.ones(147)
    #kernel = np.ones(72)
    closed = cv2.morphologyEx(thres, cv2.MORPH_CLOSE, kernel)
    # erosion and dilation
    closed = cv2.erode(closed, None, iterations = 5)
    closed = cv2.dilate(closed, None, iterations = 5)

    contours, hierarchy = cv2.findContours(closed, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

    # finding the largest contour
    max_cont = 0
    index = 0
    size = len(contours)

    for i,j in zip(contours,range(size)):
        area= cv2.contourArea(i)
        x = int(area)
        if(x>max_cont):
          max_cont = x
          index = j

    print(max_cont)
    print(index)
    # draw a bounding box arounded the detected barcode and display the
    # image
    cv2.drawContours(img, contours[index], -1, (255,0,255), 3)

    cv2.imshow("Image", img)
    #cv2.waitKey(0)

    # displaying images 
    
    #cv2.imshow('dilated img', dil)
    #cv2.imshow('eroded img', erode)
    
    #cv2.imshow('opening',opening)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [12]:
def detection():
    # variables
    images = []
    resized = []
    gray = []
    edge = []
    threashold_image = []
    path=root.filen[0]
    img = cv2.imread(path)
    img2 = img.copy()
    images.append(img)

    # resizing images
    #resized_img = cv2.resize(img,(500,500))
    #resized.append(resized_img)

    grayImage = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
    gray.append(grayImage)

    # calculating edges using canny edge detection
    new_img = (grayImage*255).astype(np.uint8)
    edges = cv2.Canny(new_img,100,200)
    edge.append(edges)

    # gausian thresholding
    thres = adaptive_gaussian_thresholding(edges)
    threashold_image.append(thres)

    # binarize the image
    bin = cv2.threshold(thres, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

    # performing morphological operations 
    # construct a closing kernel containing all ones to perform operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))          ## np.ones(147)
    #kernel = np.ones(72)
    closed = cv2.morphologyEx(thres, cv2.MORPH_CLOSE, kernel)
    # erosion and dilation
    closed = cv2.erode(closed, None, iterations = 5)
    closed = cv2.dilate(closed, None, iterations = 5)

    contours, hierarchy = cv2.findContours(closed, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

    # finding the largest contour
    max_cont = 0
    index = 0
    size = len(contours)

    for i,j in zip(contours,range(size)):
        area= cv2.contourArea(i)
        x = int(area)
        if(x>max_cont):
          max_cont = x
          index = j

    print(max_cont)
    print(index)
    # draw a bounding box arounded the detected barcode and display the
    # image
    cv2.drawContours(img, contours[index], -1, (255,0,255), 3)

    
    #cv2.waitKey(0)

    # displaying images 
    
   
    cv2.imshow('grey img', grayImage)
    cv2.waitKey(0)
    cv2.imshow('edge', edges)
    cv2.waitKey(0)
    cv2.imshow('threshold',thres)
    cv2.waitKey(0)
    cv2.imshow('binary', bin)
    cv2.waitKey(0)
    
    
    
    #cv2.imshow('dilated img', dil)
    #cv2.imshow('eroded img', erode)
    cv2.imshow('closing',closed)
    #cv2.imshow('opening',opening)
    cv2.waitKey(0)
    cv2.imshow("Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [13]:

def inputpic():
    
    root.filen=filedialog.askopenfilenames(initialdir=os.getcwd(),title="select image",filetypes=[('Image','*.jpg *.jpeg *.png')])
    imagee=Image.open(root.filen[0])
    imagee=imagee.resize((400,400),Image.ANTIALIAS)
    imagee = ImageTk.PhotoImage(imagee)
    #img=Image.open(filen)
    #img=ImageTk.PhotoImage(img)
    lbl_pic_path.configure(image=imagee)
    lbl_pic_path.image=imagee
    

In [14]:

root=Tk()

frame=Frame(root)
frame.pack(side=BOTTOM,padx=15,pady=15)

lbl_pic_path =Label(root)
lbl_pic_path.pack()

btn_browse=Button(frame,text="select image",command=inputpic)
btn_browse.pack(side=tk.LEFT)
btn_browse2=Button(frame,text="Show Results",command=detectionResult)
btn_browse2.pack(side=tk.LEFT,padx=12)
btn_browse3=Button(frame,text="Show Steps",command=detection)
btn_browse3.pack(side=tk.LEFT,padx=9)
root.title("selectimage")
root.geometry("400x450")
root.mainloop()

C:\Users\eisha\AppData\Local\Temp\ipykernel_25980\4205945076.py:5: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  imagee=imagee.resize((400,400),Image.ANTIALIAS)


62630
15
62630
15
62630
15
